读取TXT文件，统计文件字符数并打印前100个字符

In [1]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:    
    raw_text = f.read() 

print("Total number of character:", len(raw_text)) 
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


测试正则语法库re

In [3]:
import re 

text = "Hello, world. This, is a test." 
result = re.split(r'(\s)', text) 

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


修改空格（\s）、逗号和句点（[,.]）上的正则表达式拆分

In [4]:
result = re.split(r'([,.]|\s)', text) 

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


安全删除冗余空格字符

In [5]:
result = [item for item in result if item.strip()] 

print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


处理更多类型的字符

In [6]:
text = "Hello, world. Is this-- a test?" 

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)

result = [item.strip() for item in result if item.strip()] 

print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


使用简单编码器编码整篇小说

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) 

preprocessed = [item.strip() for item in preprocessed if item.strip()] 

print(len(preprocessed))

4690


打印查看前30个tokens

In [9]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


将编码后的tokens映射为唯一整数（token ID）

In [11]:
all_words = sorted(set(preprocessed)) 

vocab_size = len(all_words) 

print(vocab_size)

1130


创建词汇表并打印前51个条目

In [12]:
vocab = {token:integer for integer,token in enumerate(all_words)} 

for i, item in enumerate(vocab.items()):    
    print(item)    
    if i >= 50: 
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


实现一个简单的文本tokenizer

In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab # 将词汇表存储为类属性，以便在编码和解码方法中访问
        self.int_to_str = {i:s for s,i in vocab.items()} # 创建一个逆向词汇表，将token ID 映射回原始文本 tokens
    
    # 将输入文本处理成token IDs
    def encode(self, text):
        # 分词
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        # 处理空格
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        # 使用词汇表将tokens映射为token IDs
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    # 将token IDs映射回文本
    def decode(self, ids):
        # 将ids中的tokenIDs通过词汇表反向映射并连接起来
        text = " ".join([self.int_to_str[i] for i in ids])
        # 删除指定标点符号前的空格
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text            

实例化tokenizer对象并进行编解码

In [15]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
    Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [16]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


使用文中未出现过的词进行编码会报错

In [17]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

加入两个特殊标记

In [18]:
all_tokens = sorted(list(set(preprocessed))) 

all_tokens.extend(["<|endoftext|>", "<|unk|>"]) 

vocab = {token:integer for integer,token in enumerate(all_tokens)} 

print(len(vocab.items()))

1132


查看加入的两个特殊标记

In [19]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


可以处理未知单词的tokenizer

In [20]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab # 将词汇表存储为类属性，以便在编码和解码方法中访问
        self.int_to_str = {i:s for s,i in vocab.items()} # 创建一个逆向词汇表，将token ID 映射回原始文本 tokens
    
    # 将输入文本处理成token IDs
    def encode(self, text):
        # 分词
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        # 处理空格
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        # 将未知单词替换为 <|unk|>字符
        preprocessed = [item if item in self.str_to_int 
                        else "<|unk|>" for item in preprocessed]
        # 使用词汇表将tokens映射为token IDs
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    # 将token IDs映射回文本
    def decode(self, ids):
        # 将ids中的tokenIDs通过词汇表反向映射并连接起来
        text = " ".join([self.int_to_str[i] for i in ids])
        # 删除指定标点符号前的空格
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text            

测试新的tokenizer

In [21]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [22]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [23]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
